In [1]:
import torch

def f(x):
    return 2*x[0]**2 + 3*x[1]

def df_tangent(x):
    return 4*x[0], 3

def df_adjoint(x_input):
    x = torch.tensor(x_input, dtype=torch.float32, requires_grad=True)

    y = 2*x[0]**2 + 3*x[1]
    y.backward()
    return x.grad

x = (2.0, 3.0)

print(f"Function value f(x): {f(x)}")
print(f"Manual Gradient (Tangent): {df_tangent(x)}")
print(f"Auto-diff Gradient (Adjoint): {df_adjoint(x).tolist()}")

Function value f(x): 17.0
Manual Gradient (Tangent): (8.0, 3)
Auto-diff Gradient (Adjoint): [8.0, 3.0]


In [45]:
from finmarkets import generate_dates, SwapSide
from finmarkets import GlobalConst

from itertools import pairwise

class InterestRateSwap:
    def __init__(self, nominal, start_date, maturity, fixed_rate, frequency_float, frequency_fix="1y", side=SwapSide.Receiver):
        self.nominal = nominal
        self.fixed_rate = fixed_rate
        self.fix_dates = generate_dates(start_date, maturity, frequency_fix)
        self.float_dates = generate_dates(start_date, maturity, frequency_float)
        self.side = side

    def dv01_tangent_mode(self, dc, dr=0.0001):
        temp = 0.0
        for d0, d1 in pairwise(self.fix_dates):
            tau = (d1-d0).days / 360
            t = (d1 - GlobalConst.OBSERVATION_DATE).days / 365
            temp += -t * tau * dc.df(d1)
        dfixed_pv = self.nominal * self.fixed_rate * dr * temp

        temp = 0.0
        for d0, d1 in pairwise(self.float_dates):
            tau = (d1-d0).days / 365
            t_start = (d0 - self.float_dates[0]).days / 365
            t_end = (d1 - self.float_dates[0]).days / 365

            P_start = dc.df(d0)
            P_end = dc.df(d1)
            F = (P_start/P_end - 1)/tau

            dF_dr = (t_end - t_start) * (P_start / (tau * P_end))
            term = (dF_dr * P_end) + (F * (-t_end * P_end)) / tau
            temp += term

        dfloat_pv = self.nominal * dr * temp
        print (dfixed_pv, dfloat_pv)
        return self.side*(dfixed_pv - dfloat_pv)
    
    def npv(self, dc):
        temp = sum([dc.df(d1) * (d1-d0).days / 365 for d0, d1 in pairwise(self.fix_dates)])
        fixed_pv = self.nominal * self.fixed_rate * temp

        temp = 0.0
        for d0, d1 in pairwise(self.float_dates):
            tau = (d1-d0).days / 365
            P_start = dc.df(d0)
            P_end = dc.df(d1)
            F = (P_start/P_end - 1)/tau          
            temp += F * tau * dc.df(d1)
        float_pv = self.nominal * temp
        return self.side*(fixed_pv - float_pv)

    # def bpv(self, dc):
    #     return 0.0001*self.annuity(dc)

    # def swap_rate_single_curve(self, dc):
    #     den = 0
    #     num = dc.df(self.fix_dates[0]) - dc.df(self.fix_dates[-1])
    #     for i in range(1, len(self.fix_dates)):
    #         tau = (self.fix_dates[i]-self.fix_dates[i-1]).days/360
    #         den += dc.df(self.fix_dates[i])*tau
    #     return num/den

In [46]:
import numpy as np

from finmarkets import TimeInterval, DiscountCurve#, FlatTermStructure

years = 5
dates = [GlobalConst.OBSERVATION_DATE + TimeInterval(f"{i}y") for i in range(years+1)]
rates = np.array([0.015]*(years+1))
dc = DiscountCurve(dates, np.exp(-rates*np.arange(0, years+1)))

fixed_rate = 0.05

swap = InterestRateSwap(1e06, GlobalConst.OBSERVATION_DATE, f"{years}y", fixed_rate, "1y", "3m", SwapSide.Receiver)
print (swap.npv(dc))
print (swap.dv01_tangent_mode(dc))

168278.84737204766
-63.292502486597684 463.88371854781843
-527.1762210344161


In [48]:
import numpy as np

from finmarkets import TimeInterval, DiscountCurve, FlatTermStructure

years = 5
dates = [GlobalConst.OBSERVATION_DATE + TimeInterval(f"{i}y") for i in range(years+1)]
rates = np.array([0.01501]*(years+1))
dc = DiscountCurve(dates, np.exp(-rates*np.arange(0, years+1)))
fixed_rate = 0.05

swap = InterestRateSwap(1e06, GlobalConst.OBSERVATION_DATE, f"{years}y", fixed_rate, "1y", "3m", SwapSide.Receiver)
print (swap.npv(dc))

168226.22231436346


In [49]:
import torch

class torch_DiscountCurve:
    def __init__(self, pillar_dates, rates):
        self.pillar_dates = pillar_dates
        self.t_pillars = torch.tensor([(p - GlobalConst.OBSERVATION_DATE).days / 365 for p in pillar_dates], dtype=torch.float32)
        # if isinstance(rates, torch.Tensor):
        self.rates = rates
        # else:
        #     self.rates = torch.tensor(rates, dtype=torch.float64, requires_grad=True)
        self.dfs = torch.exp(-self.rates * self.t_pillars)
        if 0 not in self.t_pillars:
            self.t_pillars =  torch.cat([torch.tensor([0.0], dtype=torch.float32), self.t_pillars])
            self.dfs = torch.cat([torch.tensor([1.0], dtype=torch.float32, requires_grad=True), self.dfs])
        self.log_dfs = torch.log(self.dfs)

    def df(self, adate):
        d = torch.tensor((adate - GlobalConst.OBSERVATION_DATE).days / 365)
        idx = torch.searchsorted(self.t_pillars, d)
        d_prev, d_next = self.t_pillars[idx-1], self.t_pillars[idx]
        log_df_prev, log_df_next = self.log_dfs[idx-1], self.log_dfs[idx]

        alpha = (d - d_prev) / (d_next - d_prev)
        interp_log_df = log_df_prev + alpha * (log_df_next - log_df_prev)

        return torch.exp(interp_log_df)

class torch_TermStructure:
    def __init__(self, pillars_dates, spot_rates):
        self.pillars_dates = pillars_dates
        self.t_pillars = torch.tensor([(p - GlobalConst.OBSERVATION_DATE).days / 365 for p in pillars_dates], dtype=torch.float32)

        # if not isinstance(spot_rates, torch.Tensor):
        #     self.rates = torch.tensor(spot_rates, dtype=torch.float32, requires_grad=True)
        # else:
        self.rates = spot_rates

    def interp_rate(self, adate):
        d = torch.tensor((adate - GlobalConst.OBSERVATION_DATE).days / 365.0, dtype=torch.float32)

        if d < self.t_pillars[0] or d > self.t_pillars[-1]:
            raise ValueError(f"Cannot extrapolate rates (date: {adate}).")

        idx = torch.searchsorted(self.t_pillars, d)

        if d == self.t_pillars[0]:
            return d, self.rates[0]

        d_prev = self.t_pillars[idx-1]
        d_next = self.t_pillars[idx]
        r_prev = self.rates[idx-1]
        r_next = self.rates[idx]

        alpha = (d - d_prev) / (d_next - d_prev)
        r_interp = r_prev + alpha * (r_next - r_prev)

        return d, r_interp

    def forward_rate(self, d1, d2):
        t1, r1 = self.interp_rate(d1)
        t2, r2 = self.interp_rate(d2)

        return (r2 * t2 - r1 * t1) / (t2 - t1)
    
from itertools import pairwise

from finmarkets import generate_dates, SwapSide, GlobalConst

class torch_InterestRateSwap:
    def __init__(self, nominal, start_date, maturity, fixed_rate, frequency_float, frequency_fix="1y", side=SwapSide.Receiver):
        self.nominal = torch.tensor(float(nominal))
        self.fixed_rate = torch.tensor(float(fixed_rate))
        self.fix_dates = generate_dates(start_date, maturity, frequency_fix)
        self.float_dates = generate_dates(start_date, maturity, frequency_float)
        self.side = torch.tensor(float(side))

    def annuity(self, dc):
        a = torch.tensor(0.0, dtype=torch.float64)

        for d0, d1 in pairwise(self.fix_dates):
            tau = (d1-d0).days / 365.0
            a = a + tau * dc.df(d1)
        return a

    def swap_rate(self, dc):
        num = 0.0
        for d0, d1 in pairwise(self.float_dates):        
            tau = (d1-d0).days / 365.0
            F = (dc.df(d0)/dc.df(d1) - 1)/tau            
            D = dc.df(d1)
            num = num + (F * tau * D)
        return num / self.annuity(dc)

    def npv_fixed(self, dc):
        return self.nominal * self.fixed_rate * sum([dc.df(d1)*(d1-d0).days / 365 for d0, d1 in pairwise(self.fix_dates)])

    def npv_float(self, dc):
        npv = 0.0
        for d0, d1 in pairwise(self.float_dates):
          tau = (d1-d0).days / 365
          F = (dc.df(d0)/dc.df(d1) - 1)/tau
          D = dc.df(d1)
          npv += tau * F * D
        return self.nominal * npv

    def npv(self, dc):
        return self.side * (self.npv_fixed(dc) - self.npv_float(dc))

    def npv2(self, dc):
        S = self.swap_rate(dc)
        A = self.annuity(dc)
        return self.side * self.nominal * (self.fixed_rate - S) * A

In [ ]:
import torch

from finmarkets import TimeInterval

dates = [GlobalConst.OBSERVATION_DATE + TimeInterval(f"{i}y") for i in range(7)]
rates = torch.tensor([0.015,0.015,0.015,0.015,0.015,0.015,0.015], dtype=torch.float32, requires_grad=True)
dc = torch_DiscountCurve(dates, rates)
fixed_rate = 0.05

torch_swap = torch_InterestRateSwap(1_000_000, GlobalConst.OBSERVATION_DATE, "5y", fixed_rate, "1y", "3m")

fixed_val = torch_swap.npv_fixed(dc)
fixed_val.backward(retain_graph=True)
delta_fixed = rates.grad.clone()*0.0001
rates.grad.zero_()

float_val = torch_swap.npv_float(dc)
float_val.backward(retain_graph=True)
delta_float = rates.grad.clone()*0.0001

print("Fixed Leg Delta:", delta_fixed.sum())
print("Floating Leg Delta:", delta_float)

rates.grad.zero_()
current_npv = torch_swap.npv(dc)
print (current_npv)
current_npv.backward(retain_graph=True)
print (rates.grad.clone().sum()*0.0001)

Fixed Leg Delta: tensor(-62.4235)
Floating Leg Delta: tensor([ 0.0000e+00, -6.2500e-06,  0.0000e+00,  0.0000e+00, -2.5017e-05,
         4.6411e+02,  0.0000e+00])
tensor(168235.7344, grad_fn=<MulBackward0>)
tensor(-526.5303)


In [57]:
rates.grad.zero_()
current_npv = torch_swap.npv2(dc)
current_npv.backward()
print (current_npv)
print (rates.grad.clone().sum()*0.0001)

tensor(168235.7079, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(-526.5304)
